<a href="https://colab.research.google.com/github/PHATTARAWAT2312/ConcealedGun/blob/main/2_Tracking_Model_Predict_10Frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tracking Model Predict-10Frame 
- นำผลลัพธ์จาก FasterRCNN Model มา Tracking ทีละ 10 เฟรม
จะได้ 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
#root_path = 'gdrive/My Drive/ConcealedGunPROJECT2020/'  #change dir to your project folder

Mounted at /content/gdrive


In [2]:
# import all the tools we need
import urllib
import pandas as pd
import numpy as np
import cv2
import itertools
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset
import os 
from PIL import Image
import random
import xml.etree.ElementTree as ET
import time
import requests
from pandas import DataFrame
import glob
import json
import subprocess
import pickle
from sklearn.preprocessing import StandardScaler
import shutil
import pathlib
from statistics import mean 
from matplotlib import pyplot as plt

### รันแล้วจะมีให้ใส่เพียง ID Clip

In [132]:
List_Test = [] 
resultdf_test0 = []
# เชื่อมคนที่เป็นคนเดียวกันในทุกเฟรม
def classify(IDX, IDFrame):
  j = IDX # คนที่ j             
  i = IDFrame  #เฟรมที่ i
  # เปรียบเทียบเฟรมที่ 0 กับ 1
  frame0 = resultdf[resultdf['idx_frame'] ==i]                     # เฟรมที่ 0 ทั้งหมด
  per0 = frame0[frame0['idx_per_next'] ==j]                      # คนที่ 0 ในเฟรมที่ 0
  per0_result = int(per0['idx_per_next'])                             # เรียกมาแค่ค่าเดียวคือ (0)

  frame1 = resultdf[resultdf['idx_frame'] ==i+1]                  # เฟรมที่ 1 ทั้งหมด
  per0n1 = frame1[frame1['idx_per'] == per0_result]           # ให้ idx_per_next ของเฟรมที่ 0 = idx_per ของเฟรมที่ 1
  per0n1_result = int(per0n1['idx_per_next'])                        # ดูผลลัพธ์
  df1=pd.concat([per0, per0n1], ignore_index=True)             # รวมตาราง

  # เปรียบเทียบเฟรมที่ 1 กับ 2
  frame2 = resultdf[resultdf['idx_frame'] ==i+2]              
  per0n2 = frame2[frame2['idx_per']==per0n1_result]
  per0n2_result = int(per0n2['idx_per_next'])         
  df2=pd.concat([df1,per0n2], ignore_index=True)

  # เปรียบเทียบเฟรมที่ 2 กับ 3
  frame3 = resultdf[resultdf['idx_frame'] ==i+3]
  per0n3 = frame3[frame3['idx_per']==per0n2_result]
  per0n3_result = int(per0n3['idx_per_next']) 
  df3=pd.concat([df2,per0n3], ignore_index=True)

  # เปรียบเทียบเฟรมที่ 3 กับ 4
  frame4 = resultdf[resultdf['idx_frame'] ==i+4]
  per0n4 = frame4[frame4['idx_per']==per0n3_result]
  per0n4_result = int(per0n4['idx_per_next']) 
  df4=pd.concat([df3,per0n4], ignore_index=True)

  # เปรียบเทียบเฟรมที่ 4 กับ 5
  frame5 = resultdf[resultdf['idx_frame'] ==i+5]
  per0n5 = frame5[frame5['idx_per']==per0n4_result]
  per0n5_result = int(per0n5['idx_per_next']) 
  df5=pd.concat([df4,per0n5], ignore_index=True)

  # เปรียบเทียบเฟรมที่ 5 กับ 6
  frame6 = resultdf[resultdf['idx_frame'] ==i+6]
  per0n6 = frame6[frame6['idx_per']==per0n5_result]
  per0n6_result = int(per0n6['idx_per_next']) 
  df6=pd.concat([df5,per0n6], ignore_index=True)

  # เปรียบเทียบเฟรมที่ 6 กับ 7
  frame7 = resultdf[resultdf['idx_frame'] ==i+7]
  per0n7 = frame7[frame7['idx_per']==per0n6_result]
  per0n7_result = int(per0n7['idx_per_next']) 
  df7=pd.concat([df6,per0n7], ignore_index=True)

  # เปรียบเทียบเฟรมที่ 7 กับ 8
  frame8 = resultdf[resultdf['idx_frame'] ==i+8]
  per0n8 = frame8[frame8['idx_per']==per0n7_result]
  per0n8_result = int(per0n8['idx_per_next']) 
  df8=pd.concat([df7,per0n8], ignore_index=True)

  # เปรียบเทียบเฟรมที่ 8 กับ 9
  frame9 = resultdf[resultdf['idx_frame'] ==i+9]
  per0n9 = frame9[frame9['idx_per']==per0n8_result]
  per0n9_result = int(per0n9['idx_per_next']) 
  df9 = pd.concat([df8,per0n9], ignore_index=True)
  List_Test.append(df9)
  return  df9

test = pd.read_csv ('/content/gdrive/MyDrive/CONEX/CONEX_ConcealedGun/DATASET.csv')
group_per = test.groupby(['Clip_Name','person_id','people']).size().reset_index(name='counts_frame')
idclip = str(input("ID Clip : "))
# Ex. QP57keNgs4o_dup1
idclip_csv = idclip+'.csv'
idclip_csv
resultdf = pd.read_csv ('/content/gdrive/MyDrive/CONEX/CONEX_ConcealedGun/Test_Predice_New/'+idclip_csv)
# resultdf = resultdf.groupby(['idx_frame']).size().reset_index(name='counts_frame')
resultdf

asd = group_per[group_per['Clip_Name'] ==idclip]
Num_frame= asd['counts_frame'].agg(['max'])
Num_frame = int(Num_frame) 
# print(Num_frame)***

per_counts = asd[asd['counts_frame'] ==Num_frame]
b = 10
Num_frame0 = Num_frame//b 
# print(Num_frame0)

Num_frame1 = []
n = 0
while n <= Num_frame0-1:
    k = 10 * n
    n += 1
    Num_frame1.append (k)
#print(Num_frame1)***
for p in range(len(Num_frame1)): # จำนวนเฟรมว่ามีกี่ช่วง (%10)
  # print(p) 
  for  m in range(len(set(resultdf['idx_per']))): # จำนวนคน
    #print(m)
    df= classify(IDX=0, IDFrame=p*10) 
    ClassMode, ProbMean = [], []
    [ClassMode.append(df['pred'].mode().tolist()[0]) for i in range(10)]
    [ProbMean.append(df['prob'].mean()) for i in range(10)]
    df['VideoName'] = idclip
    df['PersonNo'] = m 
    df['ClassMode'] = ClassMode
    df['ProbMean'] = ProbMean
    if p == Num_frame1:   # ให้ข้ามได้
      continue
    resultdf_test0.append(df)
# print(resultdf_test0[:])
# print(resultdf_test0)
all_df = pd.concat(resultdf_test0)
all_df
os.chdir('/content/gdrive/MyDrive/CONEX/CONEX_ConcealedGun/Result_Test_Tracking')
df_f.to_csv(idclip+'.csv', index = False) 

ID Clip : QP57keNgs4o_dup1


,idx_frame,idx_per,idx_per_next,test_img,test_box,prob,pred,VideoName,PersonNo,ClassMode,ProbMean
0,0,0,0,[[[0.7607843 0.78431374 0.7764706 ]\n [0.756...,[ 851.3659 281.56195 1127.345 720. ],0.993495,2,QP57keNgs4o_dup1,0,2,0.992104
1,1,0,0,[[[0.77254903 0.79607844 0.7882353 ]\n [0.764...,[ 840.2762 278.33243 1134.3063 720. ],0.995745,2,QP57keNgs4o_dup1,0,2,0.992104
2,2,0,0,[[[0.7647059 0.7882353 0.78039217]\n [0.756...,[ 641.86334 313.0655 1094.4243 720. ],0.979610,2,QP57keNgs4o_dup1,0,2,0.992104
3,3,0,0,[[[0.7647059 0.7882353 0.78039217]\n [0.756...,[ 769.14197 299.09 1086.4109 720. ],0.996243,2,QP57keNgs4o_dup1,0,2,0.992104
4,4,0,0,[[[0.7647059 0.7882353 0.78039217]\n [0.756...,[ 790.2046 293.1492 1092.9125 720. ],0.996498,2,QP57keNgs4o_dup1,0,2,0.992104
...,...,...,...,...,...,...,...,...,...,...,...
5,55,0,0,[[[0.7921569 0.8117647 0.7882353 ]\n [0.788...,[ 794.7487 289.13406 1084.0594 720. ],0.993917,2,QP57keNgs4o_dup1,1,2,0.991397
6,56,0,0,[[[0.7921569 0.8117647 0.7882353 ]\n [0.788...,[ 811.8642 293.42752 1097.0814 718.90894],0.990314,2,QP57keNgs4o_dup1,1,2,0.991397
7,57,0,0,[[[0.7921569 0.8117647 0.7882353 ]\n [0.788...,[ 805.05963 291.10287 1094.0864 720. ],0.994047,2,QP57keNgs4o_dup1,1,2,0.991397
8,58,0,0,[[[0.7921569 0.8117647 0.7882353 ]\n [0.788...,[ 807.8431 289.4305 1101.1409 720. ],0.993354,2,QP57keNgs4o_dup1,1,2,0.991397


### ได้คำตอบออกมาทีละ 10 เฟรม 
จากตัวอย่าง QP57keNgs4o_dup1
ได้ 120 เฟรม (2 คน* 60 เฟรม)

# Completed ผลลัพธ์ที่ได้ คือ
1. PersonNo	ที่ทำการ Tracking ระบุว่าเป็นคนเดียวกันแล้ว
2. ClassMode ของคนนั้นทั้งคลิปวิดีโอ
3. ProbMean เป็น Scor ที่ทำนายว่าเป็น Class นั้น